# Introduction to Linear Algebra with Python
### Matias Bayas-Erazo

This notebook introduces the functions and methods available in Python to do simple vector and matrix computations.

#### 1. Inner Product and Norm:

Recall that the inner produce of two vectors $x,y \in R^n$ is given by:

$$ x'y = \sum_{i=1}^n x_i y_i $$

This can be easily computed as follows:

In [3]:
import numpy as np

x = np.ones(3)
y = np.array((3, 5, 7))

sum(x*y) # Computes the inner product of x and y

15.0

On the other hand the norm of a vector is, 

$$ ||x|| = \sqrt{x'x} = \left(\sum_{i=1}^n x_i^2 \right)^{1/2} $$

This can be calculated by Python using standard functions or by accesing `np.linalg`

In [4]:
np.sqrt(np.sum(x**2)) # Norm of x, version 1

1.7320508075688772

In [5]:
np.linalg.norm(x) # Norm of x, version 2

1.7320508075688772

#### 2. Matrices

NumPy arrays can also be used as matrices. For example, we can create a 2 by 2 matrix as follows:

In [10]:
A = np.ones((2,2))
A.shape

(2, 2)

The shape attribute is a tuple giving the number of rows and columns. To get the transpose of A simply use `A.transpose` or `A.T`. Matrix operations are very simple as well. In the following, I create an identity matrix and then show how matrix addition, scalar multiplication, and matrix multiplication can be implemented in Python.

In [14]:
B = np.identity(2)
2*B # Scalar multiplication

array([[ 2.,  0.],
       [ 0.,  2.]])

In [15]:
A + B # Matrix addition

array([[ 2.,  1.],
       [ 1.,  2.]])

In [16]:
A @ B # Matrix multiplication

array([[ 1.,  1.],
       [ 1.,  1.]])

We have to be careful because `A*B` is not the product of matrices A and B. It returns the element by element product.

#### 3. Linear Equations with SciPy

The following is an illustration of how to solve linear equations with SciPy's `linalg` subpackage. The idea is to use methods that allow us to find a solution to system of equations that have the following form:

$$ y = Ax $$


In [25]:
import numpy as np
from scipy.linalg import inv, solve, det

A = np.array((1,2,3,4))
A.shape = (2,2)
y = np.ones((2,1))
det(A) # Check that A is non-singular.

-2.0

In [33]:
A_inv = inv(A) # Compute inverse

# Find solution, method 1:
x1 = A_inv @ y

# Find solution, method 2:
x2 = solve(A, y)

The second method is preferred because the algorithm is more stable. To check that we have the right solution, we can compute $Ax$, which should equal y:

In [35]:
A @ x2

array([[ 1.],
       [ 1.]])

#### 4. Eigenvalues and Eigenvectors

Using SciPy we can also solve for the eigenvalues and and eigenvectors of a matrix as follows:

In [41]:
from scipy.linalg import eig

A = ((1,2),(2,1))
A = np.array(A)
evals, evecs = eig(A)
print(evals) 
print(evecs)

[ 3.+0.j -1.+0.j]
[[ 0.70710678 -0.70710678]
 [ 0.70710678  0.70710678]]


#### 5. Projections and Orthonormal vectors

The following function computes the orthonormal vectors of of the space spanned by the linearly independent columns of X. Recall the result that states that for any linearly independent set, $(x_1, ... x_k)$, there exists a set of orthonormal vectors $(u_1, ... , u_k)$ that span the same space. Implementation uses the Gram-Schmidt orthogonalization. 

In [56]:
import numpy as np

def gram_schmidt(X):

    """
    Implements Gram-Schmidt orthogonalization.

    Parameters
    X : an n x k array with linearly independent columns

    Returns
    U : an n x k array with orthonormal columns

    """
    
    # Set up:
    n, k = X.shape
    U = np.empty((n,k))
    
    # The first column of U is a normalization of the first column of X:
    v1 = X[:,0]
    U[:,0] = v1/np.sqrt(np.sum(v1**2))
    
    # Loop for the orthogonal projections of the columns of X:
    
    for i in range(1,k):
        x = X[:,i] # Column to project
        Z = X[:, 0:i]   # first i-1 columns of X (?)
        
        M = np.eye(n) - Z@np.linalg.inv(Z.T @ Z)@Z.T
        
        # Project column:
        u =  M@x
        U[:,i] = u / np.sqrt(np.sum(u**2))

    return U
    


We will now work with some arrays and project vectors using various methods. We begin by using the projection matrix $P_X = X(X'X)^{-1}X'$ to project y onto $S_X$.

In [57]:
# Setup:
y = [1, 3, -3]

X = [[1, 0],
     [0, -6],
     [2, 2]]

X, y = [np.asarray(z) for z in (X, y)]
        
    
# Projects y into column space of X:
Py1 = X@np.linalg.inv(X.T@X)@X.T @y
Py1
    

array([-0.56521739,  3.26086957, -2.2173913 ])

Let's now try to project y using the orthonormal basis that can be computed using the function `gram_schmidt`:

In [59]:
U = gram_schmidt(X) # Gets the orthonormal vectors
Py2 = U@U.T@y # Projects
Py2


array([-0.56521739,  3.26086957, -2.2173913 ])

A third way to project $y$ is using QR decomposition:

In [63]:
from scipy.linalg import qr

Q, R = qr(X, mode='economic')

Py3 = Q@Q.T@y
Py3

array([-0.56521739,  3.26086957, -2.2173913 ])